Based on: 
* https://python.langchain.com/docs/tutorials/rag/
* https://python.langchain.com/docs/how_to/document_loader_directory/
* https://python.langchain.com/docs/integrations/document_loaders/email/

In [ ]:
import os
import shutil
from difflib import SequenceMatcher
from devtools import debug
from dotenv import load_dotenv
load_dotenv()

In [24]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, UnstructuredEmailLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
loader = DirectoryLoader(
    "./data/emails", glob="*.eml", 
    loader_cls=UnstructuredEmailLoader, 
    show_progress=True, 
    use_multithreading=True
)
docs = loader.load()

In [5]:
def is_similar(text1: str, text2: str, threshold: float = 0.8) -> bool:
    return SequenceMatcher(None, text1, text2).ratio() > threshold

def deduplicate_documents(docs):
    unique_docs = []
    
    for doc in docs:
        # Check if current doc is similar to any existing unique doc
        if not any(is_similar(doc.page_content, existing.page_content) 
                  for existing in unique_docs):
            unique_docs.append(doc)
            
    return unique_docs

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
unique_splits = deduplicate_documents(all_splits)

In [1]:
def create_or_load_vectorstore(documents, persist_directory="./chroma_db", force_recreate=False):
    """
    Creates a new vectorstore or loads an existing one from disk.
    
    Args:
        documents: List of documents to embed
        persist_directory: Directory to save/load the vectorstore
        force_recreate: If True, recreates the vectorstore even if it exists
        
    Returns:
        Chroma: The vectorstore instance
    """
    # Initialize embedding function
    embedding_function = OpenAIEmbeddings()
    
    # Check if directory exists and contains a valid Chroma database
    has_valid_store = (
        os.path.exists(persist_directory) and
        os.path.exists(os.path.join(persist_directory, "chroma.sqlite3"))
    )
    
    if has_valid_store and not force_recreate:
        print("Loading existing vectorstore...")
        try:
            vectorstore = Chroma(
                persist_directory=persist_directory,
                embedding_function=embedding_function
            )
            # Attempt to get collection stats to verify it's working
            try:
                collection_stats = vectorstore._collection.count()
                if collection_stats == 0:
                    print("Warning: Existing vectorstore is empty. Creating new one...")
                    has_valid_store = False
            except Exception:
                print("Warning: Could not verify vectorstore content. Creating new one...")
                has_valid_store = False
                
        except Exception as e:
            print(f"Error loading existing vectorstore: {e}")
            print("Creating new vectorstore instead...")
            has_valid_store = False
            
    if not has_valid_store or force_recreate:
        print("Creating new vectorstore...")
        # Clean up any existing invalid or partial data
        if os.path.exists(persist_directory):
            shutil.rmtree(persist_directory)
            
        # Create new vectorstore
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embedding_function,
            persist_directory=persist_directory
        )
        
    return vectorstore

In [19]:
store = create_or_load_vectorstore(unique_splits)

Creating new vectorstore...


In [33]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 20})

retrieved_docs = retriever.invoke("Welke problemen zijn er met CP20?")

len(retrieved_docs)

20

In [25]:
llm = ChatOpenAI(model="gpt-4")
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
rag_chain.invoke("Welke problemen zijn er met CP20?")

'De problemen met CP20 betreffen de opvoerhoogte die handmatig gewijzigd moest worden van 1m naar 13.9m om de flow in de appartementen te herstellen. Bovendien, als CP20 spanningsloos is geweest, zet het systeem de setpoint automatisch op 0% in plaats van de laatst bekende waarde. Dit zijn de voornaamste problemen die uit de gegeven context naar voren komen.'

In [35]:
rag_chain.invoke("Welke problemen zijn er met CP19?")

'De problemen met CP19 zijn niet specifiek genoemd in de tekst. Echter, er wordt wel vermeld dat de Wilo pompen (CP05 tot CP19) een foutmelding geven bij een te laag setpoint in de WarmtePompManager (te hoge druk). Een ander probleem dat wordt genoemd, maar niet specifiek aan CP19 is gekoppeld, is dat de modbus adressen van de WarmtePomp nr4 en 6 soms omdraaien bij het opstarten zonder spanning.'

In [ ]:
rag_chain.invoke("Welke werkzaamheden zijn er geweest op LH 65")

In [ ]:
rag_chain.invoke("Welke LH adressen hebben allemaal last bij problemen met CP22 en geef een korte omschrijving van de problemen")

In [ ]:
debug(retriever.invoke("Welke LH adressen hebben allemaal last bij problemen met CP22?"))

In [ ]:
rag_chain.invoke("Welke werkzaamheden zijn er geweest op LH 99")

In [ ]:
debug(retriever.invoke("Welke werkzaamheden zijn er geweest op LH 99"))

In [39]:
rag_chain.invoke("Bij welk apartement mist er een display afleverset?")

'Het is niet expliciet vermeld in de tekst welk appartement een display afleverset mist.'

In [ ]:
rag_chain.invoke("Welke informatie heb je rondom werkorder 101326480?")